# Features
Compute the features

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts
from phoenix.common import utils
from phoenix.tag import normalise
from phoenix.tag import feature
from phoenix.tag import export

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parametrise the run execution date.
# Format of the run date
RUN_DATE_FORMAT = "%Y-%m-%d"
# This can be overwritten at execution time by Papermill to enable historic runs and backfills etc.
RUN_DATE = datetime.datetime.today().strftime(RUN_DATE_FORMAT)

# Set Artefacts URL
ARTIFACTS_BASE_URL = f"{artifacts.urls.get_local()}{RUN_DATE}/"

# Input
FOR_TAGGING_ARTIFACTS_FOLDER = f"{ARTIFACTS_BASE_URL}for_tagging/"

In [ ]:
# Display params.
print(
ARTIFACTS_BASE_URL,
FOR_TAGGING_ARTIFACTS_FOLDER,
RUN_DATE,
sep='\n',
)

In [ ]:
%env DASK_CLUSTER_IP=tcp://127.0.0.1:40861

In [ ]:
utils.dask_global_init()

In [ ]:
all_items = normalise.merge(FOR_TAGGING_ARTIFACTS_FOLDER)

In [ ]:
all_items.index

In [ ]:
all_items_normalised = normalise.execute(all_items)

In [ ]:
all_items_normalised.head()

In [ ]:
lang_dist_all = normalise.language_distribution(all_items_normalised)
lang_dist_all

In [ ]:
features_df = feature.features(all_items_normalised)

In [ ]:
ex_features_df = feature.explode_features(features_df)

In [ ]:
ex_features_df.head()

In [ ]:
ex_features_df.index

In [ ]:
items, key_items, ex_features_has_df = feature.finalise(all_items_normalised, ex_features_df)

In [ ]:
items.head()

In [ ]:
key_items.head()

In [ ]:
key_items.shape

In [ ]:
a = artifacts.dataframes.persist(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "all_features"), export.get_all_features_for_export(ex_features_has_df))

In [ ]:
a = artifacts.dataframes.persist(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "key_items"), export.get_items_for_export(key_items))

In [ ]:
a = artifacts.dataframes.persist(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "items"), export.get_items_for_export(items))

In [ ]:
export.features_for_labeling(ARTIFACTS_BASE_URL, ex_features_has_df, None)

In [ ]:
export.features_for_labeling(ARTIFACTS_BASE_URL, ex_features_has_df, "tweets")

In [ ]:
export.features_for_labeling(ARTIFACTS_BASE_URL, ex_features_has_df, "facebook_posts")

In [ ]:
export.features_for_labeling(ARTIFACTS_BASE_URL, ex_features_has_df, "key_tweets")

In [ ]:
export.features_for_labeling(ARTIFACTS_BASE_URL, ex_features_has_df, "key_facebook_posts")